# Setup

In [1]:
import numpy as np
import tensorflow as tf
import os

%matplotlib inline 
import matplotlib
import matplotlib.pyplot as plt

# Basic RNNs

## Manual RNN

In [2]:
tf.reset_default_graph()

n_inputs = 3
n_neurons = 5

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

Wx = tf.Variable(tf.random_normal(shape=[n_inputs, n_neurons], dtype=tf.float32))
Wy = tf.Variable(tf.random_normal(shape=[n_neurons, n_neurons], dtype=tf.float32))
b = tf.Variable(tf.zeros([1, n_neurons], dtype=tf.float32))

Y0 = tf.tanh(tf.matmul(X0, Wx) + b)
Y1 = tf.tanh(tf.matmul(X1, Wx) + tf.matmul(Y0, Wy) + b)

init = tf.global_variables_initializer()

In [3]:
X0_batch = np.array([[0,1,2], [3,4,5], [6,7,8], [9,0,1]])
X1_batch = np.array([[9,8,7], [0,0,0], [6,5,4], [3,2,1]])

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [4]:
print(Y0_val, "\n", Y1_val)

[[ 0.90016013  0.55060285  0.97605944  0.5975035   0.6712791 ]
 [ 0.9993303   0.7418787   1.          0.999327    0.43933737]
 [ 0.99999577  0.8591379   1.          0.99999917  0.12902817]
 [-0.74217856 -0.7462043   1.          0.99479586 -0.9998391 ]] 
 [[ 0.9999991   0.99487734  1.          0.9999995  -0.99780756]
 [ 0.9214678   0.9740525  -0.99028873 -0.7395081  -0.74186856]
 [ 0.9998893   0.95619625  1.          0.9999576  -0.9629404 ]
 [ 0.99848247 -0.91935927  1.          0.99999255  0.96208984]]


# Using static_rnn()

In [5]:
n_inputs = 3
n_neurons = 5

In [6]:
tf.reset_default_graph()

X0 = tf.placeholder(tf.float32, [None, n_inputs])
X1 = tf.placeholder(tf.float32, [None, n_inputs])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, [X0, X1], dtype=tf.float32)

Y0, Y1 = output_seqs

In [7]:
init = tf.global_variables_initializer()

In [8]:
X0_batch = np.array([[0,1,2], [3,4,5], [6,7,8], [9,0,1]])
X1_batch = np.array([[9,8,7], [0,0,0], [6,5,4], [3,2,1]])

with tf.Session() as sess:
    init.run()
    Y0_val, Y1_val = sess.run([Y0, Y1], feed_dict={X0: X0_batch, X1: X1_batch})

In [9]:
print(Y0_val, "\n", Y1_val)

[[-0.9222669   0.4350022  -0.2167106   0.95406216 -0.43947324]
 [-0.99998593  0.9619077   0.93534905  0.9993183  -0.8687767 ]
 [-1.          0.99808687  0.99856424  0.99999005 -0.9749921 ]
 [-0.9999232   0.9893937   0.99997187 -0.999703   -0.9967654 ]] 
 [[-1.          0.99878347  0.9999946   0.99975854 -0.9785902 ]
 [ 0.11347174 -0.36026895 -0.3128892  -0.8332021  -0.47020394]
 [-0.9999982   0.9712861   0.9986304   0.7778055  -0.9463622 ]
 [-0.9913593   0.94167817  0.88552314 -0.6493316  -0.71470994]]


In [10]:
from tensorflow_graph_in_jupyter import show_graph

In [11]:
show_graph(tf.get_default_graph())

# Packing sequence

In [12]:
n_steps = 2
n_inputs = 3
n_neurons = 5

In [13]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
X_seqs = tf.unstack(tf.transpose(X, perm=[1, 0, 2]))

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
output_seqs, states = tf.contrib.rnn.static_rnn(basic_cell, X_seqs, dtype=tf.float32)
outputs = tf.transpose(tf.stack(output_seqs), perm=[1, 0, 2])

In [14]:
init = tf.global_variables_initializer()

In [15]:
X_batch = np.array([
    [[0, 1, 2], [9, 8, 7]],
    [[3, 4, 5], [0, 0, 0]],
    [[6, 7, 8], [6, 5, 4]],
    [[9, 0 ,1], [3, 2, 1]]
])

with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})

In [16]:
print(outputs_val)

[[[-0.7524973   0.02593591 -0.04120102 -0.22740965 -0.89709145]
  [-0.3503001  -0.49003336  0.99996376 -0.99999255 -1.        ]]

 [[-0.8443413   0.01700797  0.96882814 -0.97714454 -0.99999076]
  [-0.56658983 -0.8696566  -0.9592535  -0.12827586  0.6762943 ]]

 [[-0.9039711   0.00807721  0.9995385  -0.9995753  -1.        ]
  [-0.18629539 -0.88308305  0.9788871  -0.99919266 -0.9999986 ]]

 [[ 0.97150403  0.97691095  0.99998814 -0.99543935 -0.9999814 ]
  [ 0.08953145 -0.9222234   0.824273   -0.7240865  -0.9775633 ]]]


In [17]:
print(np.transpose(outputs_val, axes=[1, 0, 2])[1])

[[-0.3503001  -0.49003336  0.99996376 -0.99999255 -1.        ]
 [-0.56658983 -0.8696566  -0.9592535  -0.12827586  0.6762943 ]
 [-0.18629539 -0.88308305  0.9788871  -0.99919266 -0.9999986 ]
 [ 0.08953145 -0.9222234   0.824273   -0.7240865  -0.9775633 ]]


# Using dynamic_rnn()

In [18]:
n_steps = 2
n_inputs = 3
n_neurons = 5

In [19]:
tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

In [20]:
X_batch = np.array([
    #step0      #step1  
    [[0, 1, 2], [9, 8, 7]], #instance1
    [[3, 4, 5], [0, 0, 0]], #instance2
    [[6, 7, 8], [6, 5, 4]], #instance3
    [[9, 0 ,1], [3, 2, 1]], #instance4
])

init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    outputs_val = outputs.eval(feed_dict={X: X_batch})

In [21]:
print(outputs_val)

[[[-0.06835317  0.17118517  0.25777498 -0.04893581 -0.00256663]
  [ 0.92044795 -0.9999655   0.9706165  -0.95263547 -0.9866857 ]]

 [[ 0.43912014 -0.91623574  0.80768013 -0.6004069  -0.6818027 ]
  [-0.6445337   0.09086835  0.2989456  -0.6206576   0.763339  ]]

 [[ 0.766068   -0.99729913  0.9623599  -0.87133354 -0.93053734]
  [ 0.38215375 -0.99864656  0.9507821  -0.9659217  -0.44633684]]

 [[ 0.06793037 -0.98864704  0.86586416 -0.9975737   0.10537649]
  [-0.23901778 -0.9744172   0.8837709  -0.9584182   0.36431602]]]


In [22]:
show_graph(tf.get_default_graph())

# setting the sequence lengths

In [23]:
n_steps = 2
n_inputs = 3
n_neurons = 5

tf.reset_default_graph()

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)

In [24]:
seq_length = tf.placeholder(tf.int32, [None])
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32, sequence_length=seq_length)

In [25]:
init = tf.global_variables_initializer()

In [26]:
X_batch = np.array([
        #step0      #step1  
    [[0, 1, 2], [9, 8, 7]], #instance1
    [[3, 4, 5], [0, 0, 0]], #instance2 (padded with zero vectors)
    [[6, 7, 8], [6, 5, 4]], #instance3
    [[9, 0 ,1], [3, 2, 1]], #instance4
])

seq_length_batch = np.array([2, 1, 2, 2])

In [27]:
with tf.Session() as sess:
    init.run()
    outputs_val, states_val = sess.run([outputs, states], feed_dict={X:X_batch, seq_length: seq_length_batch})

In [28]:
print(outputs_val, "\n", "\n", states_val)

[[[ 0.55598503  0.43553102 -0.7835501   0.22638929  0.55902785]
  [-0.99826276  1.          0.9170955   0.99892426  0.61283183]]

 [[-0.09598723  0.9983664  -0.82413095  0.92276525  0.63661236]
  [ 0.          0.          0.          0.          0.        ]]

 [[-0.67483586  0.99999666 -0.8577105   0.9948973   0.70314574]
  [-0.99513674  0.99999547  0.81880116  0.95143694  0.8730332 ]]

 [[-0.99841946  0.99953294  0.99058914  0.8527053  -0.9999525 ]
  [-0.9175363   0.9428158  -0.6159373   0.81600237 -0.1708295 ]]] 
 
 [[-0.99826276  1.          0.9170955   0.99892426  0.61283183]
 [-0.09598723  0.9983664  -0.82413095  0.92276525  0.63661236]
 [-0.99513674  0.99999547  0.81880116  0.95143694  0.8730332 ]
 [-0.9175363   0.9428158  -0.6159373   0.81600237 -0.1708295 ]]


# Training a sequence classifier

In [29]:
tf.reset_default_graph()

In [30]:
tf.reset_default_graph()

n_steps = 28
n_inputs = 28
n_neurons = 150
n_outputs = 10

learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None]) 

basic_cell = tf.contrib.rnn.BasicRNNCell(num_units=n_neurons)
outputs, states = tf.nn.dynamic_rnn(basic_cell, X, dtype=tf.float32)

logits = tf.layers.dense(states, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)

loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
traning_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)                   # logits[i][y[i]]是否在logits[i]的top1个数据中
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))  # mean([1, 1, 0, 0, 1]) 故为准确率

init = tf.global_variables_initializer()

In [31]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_test = X_test.reshape((-1, n_steps, n_inputs))
len(X_train)

60000

In [32]:
def shuffle_batch(X, y, batch_size):
    shuffled_index = np.random.permutation(len(X))
    n_batchs = len(X) // batch_size
    for batch_idx in np.array_split(shuffled_index, n_batchs):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [33]:
n_epochs = 100
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            X_batch = X_batch.reshape((-1, n_steps, n_inputs))
            sess.run(traning_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
    print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)        

KeyboardInterrupt: 

# Multi-layer RNN

In [ ]:
tf.reset_default_graph()

n_steps = 28
n_inputs = 28
n_outputs = 10

learning_rate = 0.001

X = tf.placeholder(tf.float32, [None, n_steps, n_inputs])
y = tf.placeholder(tf.int32, [None])

In [ ]:
n_neurons = 100
n_layers = 3

layers = [tf.contrib.rnn.BasicRNNCell(num_units=n_neurons, activation=tf.nn.relu)  for layer in range(n_layers)]
multi_layer_cell = tf.contrib.rnn.MultiRNNCell(layers)
outputs, states = tf.nn.dynamic_rnn(multi_layer_cell,X, dtype=tf.float32)

In [ ]:
states_concat = tf.concat(axis=1, values=states)
logits = tf.layers.dense(states_concat, n_outputs)
xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_mean(xentropy)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(loss)
correct = tf.nn.in_top_k(logits, y, 1)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

init = tf.global_variables_initializer()

In [ ]:
n_epochs = 10
batch_size = 150

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            X_batch = X_batch.reshape(-1, n_steps, n_inputs)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test = accuracy.eval(feed_dict={X: X_test, y: y_test})
        print(epoch, "Training accuracy:", acc_train, "Test accuracy:", acc_test)

# Time series